In [ ]:
%load_ext autoreload
%autoreload 1
%aimport communicate
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import scipy.optimize

## Data Collection

In [ ]:
def collect_data(duration=5):
    """Collects log data from the cable robot.  Returns tuple of Nx4 arrays for lengths and vels"""
    with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port='/dev/tty.usbmodem100994303') as robot:
        tstart = time.time()
        while True:
            robot.update()
            time.sleep(0.01)
            if (time.time() - tstart > duration):
                break
        robot.send('d10,100')
    motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
    motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
    xy = np.array([[dat['controller'].cur_x, dat['controller'].cur_y] for dat in robot.all_data])
    return motor_ls, motor_ldots, xy

In [ ]:
if True:
    # Collect some data
    time.sleep(2)
    ls, ldots, xy = collect_data(duration=30)
    print(f'Collected {len(ls)} samples')
    print(ls[-1])
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/data.npz', ls=ls, ldots=ldots, est=xy)
else:
    with np.load('/tmp/data.npz') as f:
        ls = f['ls']
        ldots = f['ldots']
        xy = f['est']

In [ ]:
# Plot data as a sanity check
fig, axes = plt.subplots(1, 3, sharex=False, figsize=(12, 4))
axes[0].plot(ls)
axes[1].plot(ldots);
axes[2].plot(*xy.T)
axes[2].axis('equal')
axes[0].set_title('Cable Lengths (m)')
axes[1].set_title('Cable Velocities (m/s)')

## Calibration

In [ ]:
# Constants
# W, H = 2.75, 2.17
# W, H = 3.05 - .22377, 2.44 - .22377
# W, H = 3.05, 2.44
# W, H = 3.05, 2.34
# W, H = 2.2 - .22377, 2.0 - .22377
# W, H = 3.05 - 0.1778, 2.34 - 0.14
W, H = 2.9 - 0.184, 2.3 - 0.122
INIT_XS = lambda ls: np.ones(ls.shape[0] * 2) * 1.5
# INIT_LPARAMS = lambda ls: np.array([0,0,0,0,1,1,1,1,*(-ls.mean(axis=0) + 1.5)])
INIT_LPARAMS = lambda ls: np.array([0,0,0,0,1,1,1,1,*(-ls.mean(axis=0) + 1.5)])
INIT_MOUNTPOINTS = np.array([[W, 0], [W, H], [0, H], [0, 0]]).T.flatten()

# Helper functions
def l_corr(ls, params):
    params = params.reshape(-1, 4)
    # return params[0] * np.square(ls) + (1 + 0.05 * np.tanh(params[1])) * ls + params[2]
    return params[0] * np.square(ls) + ls * params[1] + params[2]
    return np.sqrt(np.square(ls*params[1] + params[2]) - params[0])
def ik(x, mountPoints):
    mountPoints = mountPoints.reshape(1, 2, 4)
    mountPoints[0, :, 3] = [0, 0]
    # mountPoints[0, :, 0] = [W, 0]
    mountPoints[0, 1, 0] = 0
    mountPoints[0, 0, 2] = 0
    mountPoints = INIT_MOUNTPOINTS.reshape(1,2,4)
    return np.sqrt(np.sum(np.square(x.reshape(-1, 2, 1) - mountPoints), axis=1))
def err(ls, params):
    N = ls.shape[0]
    mountPoints = params[:8]
    lparams = params[8:20]
    xs = params[20:20 + 2 * N].reshape(-1, 2)
    return (l_corr(ls, lparams) - ik(xs, mountPoints)).flatten()

# Actual Calibrate Function Call
def calibrate(ls, init_params = None):
    if init_params is None:
        init_params = np.concatenate((INIT_MOUNTPOINTS, INIT_LPARAMS(ls), INIT_XS(ls)))
    return scipy.optimize.least_squares(lambda params: err(ls, params),
                                        init_params,
                                        verbose=2,
                                        method='lm')

# Do the calibration
for every in (1000, 500, 100):
    INIT_XS = lambda ls: xy[::every, :].flatten()
    result = calibrate(ls[::every])
    print(result.success, result.message)
    INIT_MOUNTPOINTS = result.x[:8]  # hack to set initialization
    INIT_LPARAMS = lambda ls: result.x[8:20]
# result2 = calibrate(ls[::100])


In [ ]:
# Extract parameters and plot
mountPoints = result.x[:8].reshape(2, 4)
lparams = result.x[8:20].reshape(-1, 4)
xs = result.x[20:].reshape(-1, 2)
print('mount points:')
print(mountPoints)
print('lparams:')
print(lparams)

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 18

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(ls, ':')
plt.plot(l_corr(ls, lparams), '-')
plt.title('Cable Lengths')
plt.legend([f'cable {i} uncalibrated' for i in range(4)] + [f'cable {i} calibrated' for i in range(4)])
plt.xlabel('Data Sample \# (roughly 2ms / sample)')
plt.ylabel('Cable Length (m)')
plt.subplot(122)
plt.plot(xs[:, 0], xs[:, 1])
plt.title('Estimated Trajectory')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.axis('equal');

In [ ]:
# Output in a format that can be sent directly to the cable robot
CARRIAGE_WIDTH, CARRIAGE_HEIGHT = 0.22377, 0.22377
mountPoints2 = mountPoints * 1
mountPoints2[0, 0:2] += CARRIAGE_WIDTH
mountPoints2[1, 1:3] += CARRIAGE_HEIGHT
print(mountPoints2) # sanity check
# lparams2 = np.vstack((lparams[0], np.ones(4), lparams[1]))
lparams2 = lparams
print(lparams2) # sanity check
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',')
print('c44', *lparams2.T.flatten().tolist(), sep=',')
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',', end=';')
print('c44', *lparams2.T.flatten().tolist(), sep=',')

In [ ]:
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem100994303') as robot:
    robot.update()
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem100994303', silent=False) as robot:
    robot.send('g6')
    s1 = 'c54,' + ','.join(map(str, mountPoints2.T.flatten()))
    s2 = 'c44,' + ','.join(map(str, lparams2.T.flatten()))
    robot.send(s1)
    robot.send(s2)
    for _ in range(50):
        robot.update()